In [44]:
%load_ext autoreload
%autoreload 2

import abc
import io
import struct
from collections import defaultdict, namedtuple

import numpy as np
import read_span6_data as rs6


In [8]:
def check_header_version(buffer):
    
    if len(buffer) != 3:
        raise RuntimeError("Buffer's length is not 3 bytes!")
    
    long_start = bytes.fromhex('AA 44 12')
    short_start = bytes.fromhex('AA 44 13')
    
    header = 'UNKN'
    
    if long_start == buffer:
        header = 'LONG'
    elif short_start == buffer:
        header = 'SHORT'
        
    return header

def read_span6_header(buffer):
    header = check_header_version(buffer[0:3])
    
    header_array = ()
    header_dict = {}
    crc_len = 4
    
    if header == "LONG":
        header_array = header_long._struct.unpack(buffer[0:header_long.size])
        header_len = 28
        for num, name in enumerate(header_long._names):
            header_dict[name] = header_array[num]
            # print(f'{name} : {header_array[num]}')
            
    elif header == "SHORT":
        header_array = header_short._struct.unpack(buffer[0:header_short.size])
        header_len = 12
        
        for num, name in enumerate(header_short._names):
            header_dict[name] = header_array[num]
            # print(f'{name} : {header_array[num]}')
        
            
    else:
        raise RuntimeError("Header is unknown or buffer is not at header start")

    message_len = header_dict["message_length"]
    message_id = header_dict["message_id"]
    next_header_start = header_len + message_len + crc_len

    # if message_id == 1462:
    #     header_array = rawimusxb_message._struct.unpack(buffer[header_len:rawimusxb_message.size])
    
    return next_header_start, header_len, message_len, message_id, header_dict

def read_rawimusxb_message():
    pass



In [43]:
# def read_in_chunks(infile, chunk_size):
#     while True:
#         chunk = infile.read(chunk_size)
#         if chunk:
#             yield chunk
#         else:
#             # The chunk was empty, which means we're at the end
#             # of the file
#             return

data_span6 = r'C:\git_repo\CPT6_SST1_Converter\data\20231213124021_svVexel_1312green2.span6'

with open(data_span6, 'rb') as f1:
    buffer = f1.read()

# _, header_len, message_len, id = read_span6_header(buffer)

print('NEXT ____________________________')
# print(id)

message_ids = []
start = 0
header_dict = {}
loop = True
raw_imu_dict = {}



while loop:
    # try:
    new_start, header_len, message_len, id, hdict = read_span6_header(buffer[start:])
    message_ids.append(id)
    print(start)
    if id == 1462:
        raw_imu = rawimusxb_message._struct.unpack(buffer[start+header_len:start+header_len + rawimusxb_message.size])
        
        for num, name in enumerate(rawimusxb_message._names):
            raw_imu_dict[name] = raw_imu[num]
            print(f'{name} : {raw_imu[num]}')
            
        loop = False
                
    start = start + new_start
    # except:
    #     loop = False

gyro_scale_factor = 0.1 / (3600.0*256.0)  # rad per LSB
accel_scale_factor = 0.05/(2**15)  # m/s per LSB

print(message_ids)

def rad_to_deg(rad):
    return rad*180/np.pi


accel_z = raw_imu_dict["z_accel"] * accel_scale_factor
accel_x = raw_imu_dict["x_accel"] * accel_scale_factor
accel_y = raw_imu_dict["y_accel"] * accel_scale_factor
# A negative value implies the output is along the positive y-axis marked on the IMU
# A positive value implies the change is in the direction opposite to that of the y-axis marked on the IMU.

print(f"accel z {accel_z:.05f} m/s")
print(f"accel x {accel_x:.05f} m/s")
print(f"accel y {accel_y:.05f} m/s")

print(f"accel z {accel_z*200:.05f} m/s^2")
print(f"accel x {accel_x*200:.05f} m/s^2")
print(f"accel y {accel_y*200:.05f} m/s^2")

gyro_z = rad_to_deg(raw_imu_dict["z_gyro"] * gyro_scale_factor)
gyro_x = rad_to_deg(raw_imu_dict["x_gyro"] * gyro_scale_factor)
gyro_y = rad_to_deg(raw_imu_dict["y_gyro"] * gyro_scale_factor)
# A negative value implies the output is along the positive y-axis marked on the IMU
# A positive value implies the change is in the direction opposite to that of the y-axis marked on the IMU.

print(f"gyro z {gyro_z:.05f} deg")
print(f"gyro x {gyro_x:.05f} deg")
print(f"gyro y {gyro_y:.05f} deg")

NEXT ____________________________
0
158
imu_info : b'\x00'
imu_type : b'\x10'
gnss_week : 2292
gnss_week_seconds : 294066.6417
imu_status : 14967
z_accel : -66408
y_accel : 5676
x_accel : 82
z_gyro : -90
y_gyro : 87
x_gyro : -13
CRC : b'\xda'
[1465, 1462]
accel z -0.10133 m/s
accel x 0.00013 m/s
accel y 0.00866 m/s
accel z -20.26611 m/s^2
accel x 0.02502 m/s^2
accel y 1.73218 m/s^2
gyro z -0.00056 deg
gyro x -0.00008 deg
gyro y 0.00054 deg


In [ ]:
with open(data_span6, 'rb') as f1:
    buffer = f1.read()

_, header_len, message_len, id = read_span6_header(buffer)